In [1]:
import requests
from requests.exceptions import ConnectionError
import time
import webbrowser

from idaes.ui.fsvis.flask_server import App as fsvis_server

In [2]:
def visualize(model):
    server = fsvis_server()
    url = f"http://{server.host}:{server.port}/fs"
    
    #repeat_until_connection_available(requests.post, )
    
    #resp = requests.post(url, json={'model': str(model)}, 
     #                   params={'id': b"hello"})
        
    repeat_until_connection_available(requests.post, url, json={'model': str(model)}, 
                        params={'id': b"hello"})
    success = webbrowser.open(url)
    print(f'Opened in browser window: {success}')
    print(url)
    return server

def repeat_until_connection_available(f, *args, retries=127, **kwargs):
    for i in range(retries):
        try:
            print(f'attempt {i} of {retries}')
            return f(*args, **kwargs)
        except ConnectionError as e:
            time.sleep(0.1)
            print(f'connection error: attempt {i}; {e}')
            continue # consider logging
            
    # raise ConnectionRefusedError?? how
    # or maybe just print to debug and stop?
        


In [3]:
s = visualize('foo')

 * Serving Flask app "idaes.ui.fsvis.flask_server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Mar/2020 17:04:16] "POST /fs?id=hello HTTP/1.1" 200 -


attempt 0 of 127
connection error: attempt 0; HTTPConnectionPool(host='127.0.0.1', port=5555): Max retries exceeded with url: /fs?id=hello (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd59c4c77d0>: Failed to establish a new connection: [Errno 111] Connection refused'))
attempt 1 of 127
Opened in browser window: False
http://127.0.0.1:5555/fs


127.0.0.1 - - [04/Mar/2020 17:04:21] "GET /app HTTP/1.1" 200 -


In [1]:
s.stop() # calling this and then trying to spin up the server again seems to 
    # cause a fairly long delay before the server (re)starts??

NameError: name 's' is not defined

In [7]:
import webbrowser

webbrowser.open("http://127.0.0.1:5555/fs")

False

In [10]:
webbrowser.open("http://www.google.com", 2)

False